In [1]:
import pandas as pd
import numpy as np

In [9]:
data = pd.read_csv("C:/Users/gzsuy/Desktop/To Candidates[13236]/raw/EG Standard.csv",index_col=0)

In [10]:
data.head()

,STUDYID,DOMAIN,USUBJID,EGEVAL,EGDY,EGDTC,VISITNUM,VISIT,VISITDY,EGPOS,...,EGTEST,EGTESTCD,EGORRES,EGORRESU,EGSTRRESC,EGSTRESN,EGSTRESU,EGCAT,EGSTAT,EGSEQ
0,30121,EG,571331322||30121,INVESTIGATOR,-6,26-JAN-2011,0.0,0.0,0.0,SUPINE,...,VRMEAN,Summary (Mean) Ventricular Rate,63,63,63,63,VRMEAN,Measurement,63,1
1,30121,EG,571331323||30121,INVESTIGATOR,-23,02-JAN-2011,0.0,0.0,0.0,SUPINE,...,VRMEAN,Summary (Mean) Ventricular Rate,61,61,61,61,VRMEAN,Measurement,61,1
2,30121,EG,571331324||30121,INVESTIGATOR,1,01-JAN-2011,0.0,0.0,0.0,SUPINE,...,VRMEAN,Summary (Mean) Ventricular Rate,57,57,57,57,VRMEAN,Measurement,57,1
3,30121,EG,411331322||30121,INVESTIGATOR,-14,09-JAN-2010,0.0,0.0,0.0,SUPINE,...,VRMEAN,Summary (Mean) Ventricular Rate,86,86,86,86,VRMEAN,Measurement,86,1
4,30121,EG,411331323||30121,INVESTIGATOR,-21,23-JAN-2010,0.0,0.0,0.0,SUPINE,...,VRMEAN,Summary (Mean) Ventricular Rate,96,96,96,96,VRMEAN,Measurement,96,1


In [140]:
#preprocess - select & pivot
df = data[['USUBJID','EGDTC','EGTESTCD','EGSTAT']]
df = df[df['EGSTAT'] != 'Not done']

In [373]:
df['EGDTC']=pd.to_datetime(df['EGDTC'])
df['seq'] = df.sort_values(by=['USUBJID','EGDTC']).groupby(['USUBJID','EGTESTCD']).cumcount()+1
df["index_id"] = df[["USUBJID","seq"]].apply(lambda x:'-'.join(x.map(str)),axis=1)

df_tidy = df.pivot(index='index_id',columns='EGTESTCD',values='EGSTAT')
df_tidy['Rhythm and rate'] = np.where(df_tidy['Rhythm and rate']=='YES',1,0)
df_tidy = df_tidy.dropna(how='any')

In [374]:
#[['QTcB - Bazetts Correction Formula (Mean)','QTcF-Fridericias Correction Formul(Mean)','Summary (Mean) PR Duration','Summary (Mean) QRS Duration','Summary (Mean) QT Duration','Summary (Mean) Ventricular Rate']]
for i in df_tidy.columns:
    df_tidy[i] = df_tidy[i].map(float)
df_tidy.describe()

EGTESTCD,QTcB - Bazetts Correction Formula (Mean),QTcF-Fridericias Correction Formul(Mean),Rhythm and rate,Summary (Mean) PR Duration,Summary (Mean) QRS Duration,Summary (Mean) QT Duration,Summary (Mean) Ventricular Rate
count,454.000000,454.000000,454.000000,454.000000,454.000000,454.000000,454.000000
mean,425.872002,410.873099,0.909692,162.654185,91.185022,383.541850,75.330396
std,43.087833,38.310041,0.286939,54.752195,14.803083,41.275757,14.119987
min,165.247000,163.479000,0.000000,80.000000,40.000000,160.000000,39.000000
25%,409.719500,398.276000,1.000000,142.000000,80.250000,360.000000,65.000000
50%,427.626500,410.781000,1.000000,160.000000,88.000000,382.000000,74.000000
75%,445.198500,427.208750,1.000000,178.000000,97.000000,408.000000,83.000000
max,611.048000,559.455000,1.000000,880.000000,160.000000,514.000000,127.000000


In [375]:
df_tidy.head()

EGTESTCD,QTcB - Bazetts Correction Formula (Mean),QTcF-Fridericias Correction Formul(Mean),Rhythm and rate,Summary (Mean) PR Duration,Summary (Mean) QRS Duration,Summary (Mean) QT Duration,Summary (Mean) Ventricular Rate
index_id,,,,,,,
1001331323||30121-1,442.001,426.818,1.0,228.0,84.0,398.0,74.0
1001331325||30121-1,460.627,455.699,1.0,130.0,88.0,446.0,64.0
1001331326||30121-1,436.304,429.429,1.0,170.0,86.0,416.0,66.0
1001331327||30121-1,461.440,443.613,1.0,142.0,94.0,410.0,76.0
1001331328||30121-1,457.581,430.103,1.0,120.0,74.0,380.0,87.0


<h2>Unsupervise learning - kmeans</h2>
logic: cluster into two group and choose the smaller group and assume it as abnormal one

In [376]:
#using just numbers

In [377]:
from sklearn.cluster import KMeans

In [378]:
kmeans = KMeans(n_clusters=2,n_init=100).fit(df_tidy)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,

In [379]:
df_tidy['result'] = kmeans.labels_
df_tidy[df_tidy['result'] ==1]

EGTESTCD,QTcB - Bazetts Correction Formula (Mean),QTcF-Fridericias Correction Formul(Mean),Rhythm and rate,Summary (Mean) PR Duration,Summary (Mean) QRS Duration,Summary (Mean) QT Duration,Summary (Mean) Ventricular Rate,result
index_id,,,,,,,,
231381324||30121-1,434.349,425.382,1.0,880.0,80.0,408.0,68.0,1
481331323||30121-1,454.689,444.220,1.0,864.0,130.0,424.0,69.0,1


#research
<p>Ventricular Rate - heart rate :  60-100
<p>PR Duration upslope of the P wave to the beginning of the QRS wave: 0.12 - 0.20 sec (3 - 5 small squares) (from Nottingham University)</p>
<p>QRS Duration =0.08 - 0.12 sec (2 - 3 small squares) (from Nottingham University)</p>

<p>QT Duration: normal from 0.36 to 0.42 seconds (from ecglibrary, there are various answer but I choose this for the convenience of deciding the normal range for different calculation of QT Duration)
<p>$BazettQTc = \frac{QT}{\sqrt{2}{RR}}$ Widely used; may give erroneous results at both slow and fast heart rates</p>
<p>$Fridericia QTc = \frac{QT}{\sqrt{3}{RR}}$ Widely used; may give more consistent results at fast heart rates.</p>

In [389]:
df_tidy2 = df_tidy.copy()
df_tidy2['QTcB - Bazetts Correction Formula (Mean)'] = np.where((df_tidy2[['QTcB - Bazetts Correction Formula (Mean)']]>=360)&( df_tidy2[['QTcB - Bazetts Correction Formula (Mean)']]<=420), 1,0)
df_tidy2['QTcF-Fridericias Correction Formul(Mean)'] = np.where((df_tidy2[['QTcF-Fridericias Correction Formul(Mean)']]>=360)&( df_tidy2[['QTcF-Fridericias Correction Formul(Mean)']]<=420), 1,0)
df_tidy2['Summary (Mean) QT Duration'] = np.where((df_tidy2[['Summary (Mean) QT Duration']]>=360)&( df_tidy2[['Summary (Mean) QT Duration']]<=420), 1,0)
df_tidy2['Summary (Mean) Ventricular Rate'] = np.where((df_tidy2[['Summary (Mean) Ventricular Rate']]>=60)&( df_tidy2[['Summary (Mean) Ventricular Rate']]<=100), 1,0)
df_tidy2['Summary (Mean) QRS Duration'] = np.where((df_tidy2[['Summary (Mean) QRS Duration']]>=80)&( df_tidy2[['Summary (Mean) QRS Duration']]<=120), 1,0)
df_tidy2['Summary (Mean) PR Duration'] = np.where((df_tidy2[['Summary (Mean) PR Duration']]>=120)&( df_tidy2[['Summary (Mean) PR Duration']]<=200), 1,0)
df_tidy2['Rhythm and rate'] = np.where(df_tidy2[['Rhythm and rate']]>0.5, 1,0)
df_tidy2.head()

EGTESTCD,QTcB - Bazetts Correction Formula (Mean),QTcF-Fridericias Correction Formul(Mean),Rhythm and rate,Summary (Mean) PR Duration,Summary (Mean) QRS Duration,Summary (Mean) QT Duration,Summary (Mean) Ventricular Rate,result
index_id,,,,,,,,
1001331323||30121-1,0,0,1,0,1,1,1,0
1001331325||30121-1,0,0,1,1,1,0,1,0
1001331326||30121-1,0,0,1,1,1,1,1,0
1001331327||30121-1,0,0,1,1,1,1,1,0
1001331328||30121-1,0,0,1,1,0,1,1,0


In [392]:
kmeans = KMeans(n_clusters=2).fit(df_tidy2)
kmeans.labels_

array([1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1,

In [393]:
df_tidy2['result'] = kmeans.labels_
df_tidy2[df_tidy2['result'] == 1]

EGTESTCD,QTcB - Bazetts Correction Formula (Mean),QTcF-Fridericias Correction Formul(Mean),Rhythm and rate,Summary (Mean) PR Duration,Summary (Mean) QRS Duration,Summary (Mean) QT Duration,Summary (Mean) Ventricular Rate,result
index_id,,,,,,,,
1001331323||30121-1,0,0,1,0,1,1,1,1
1001331325||30121-1,0,0,1,1,1,0,1,1
1001331326||30121-1,0,0,1,1,1,1,1,1
1001331327||30121-1,0,0,1,1,1,1,1,1
1001331328||30121-1,0,0,1,1,0,1,1,1
231341322||30121-1,0,0,1,1,1,0,1,1
231341326||30121-1,0,0,1,1,1,1,1,1
231341328||30121-1,0,0,1,1,1,0,1,1
231341330||30121-1,0,0,1,1,1,1,1,1


In [397]:
df_tidy2['result2'] = df_tidy2.iloc[:,0:6].sum(axis=1)
df_tidy[df_tidy2.index == '231381324||30121-1']

EGTESTCD,QTcB - Bazetts Correction Formula (Mean),QTcF-Fridericias Correction Formul(Mean),Rhythm and rate,Summary (Mean) PR Duration,Summary (Mean) QRS Duration,Summary (Mean) QT Duration,Summary (Mean) Ventricular Rate,result
index_id,,,,,,,,
231381324||30121-1,434.349,425.382,1.0,880.0,80.0,408.0,68.0,1


In [399]:
df_tidy2[df_tidy2['result2'] <= 1]

EGTESTCD,QTcB - Bazetts Correction Formula (Mean),QTcF-Fridericias Correction Formul(Mean),Rhythm and rate,Summary (Mean) PR Duration,Summary (Mean) QRS Duration,Summary (Mean) QT Duration,Summary (Mean) Ventricular Rate,result,result2
index_id,,,,,,,,,
251341322||30121-1,0,0,1,0,0,0,1,1,1
271371357||30121-1,0,0,1,0,0,0,1,1,1
271381325||30121-1,0,0,0,0,0,1,1,1,1
271401322||30121-1,0,0,0,0,0,1,1,1,1
271461322||30121-1,0,0,0,0,0,0,1,1,0
271471325||30121-1,0,0,0,1,0,0,0,1,1
481331323||30121-1,0,0,1,0,0,0,1,1,1
